In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import nltk
from nltk.tokenize import word_tokenize
import sys
sys.path.append("../dataprep") 
from dataprocessing import prepare_datasets
import nltk
from nltk.tokenize import word_tokenize


In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import sys
sys.path.append("../dataprep") 
from dataprocessing import prepare_datasets

# Tải và chuẩn bị dữ liệu
train_df, test_df, val_df = prepare_datasets()

# Kết hợp train và validation để huấn luyện mô hình BiLSTM
df = pd.concat([train_df, val_df])

# Chuyển đổi nhãn thành số
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# Tokenize văn bản
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])

# Padding các sequence để có cùng độ dài
max_len = 100
X = pad_sequences(X, maxlen=max_len, padding='post')

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Định nghĩa mô hình BiLSTM
embedding_dim = 100

model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Huấn luyện mô hình
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stopping]
)

# Đánh giá mô hình trên tập test
X_test = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')
y_test = label_encoder.transform(test_df['label'])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")


Epoch 1/10
225/225 [==============================] - 30s 115ms/step - loss: 1.3042 - accuracy: 0.4633 - val_loss: 0.7015 - val_accuracy: 0.7500
Epoch 2/10
225/225 [==============================] - 27s 121ms/step - loss: 0.4719 - accuracy: 0.8392 - val_loss: 0.4563 - val_accuracy: 0.8439
Epoch 3/10
225/225 [==============================] - 26s 114ms/step - loss: 0.2369 - accuracy: 0.9213 - val_loss: 0.3837 - val_accuracy: 0.8747
Epoch 4/10
225/225 [==============================] - 26s 117ms/step - loss: 0.1442 - accuracy: 0.9544 - val_loss: 0.3630 - val_accuracy: 0.8933
Epoch 5/10
225/225 [==============================] - 29s 128ms/step - loss: 0.0987 - accuracy: 0.9689 - val_loss: 0.4048 - val_accuracy: 0.8917
Epoch 6/10
225/225 [==============================] - 28s 126ms/step - loss: 0.0788 - accuracy: 0.9758 - val_loss: 0.4009 - val_accuracy: 0.8978
Epoch 7/10
63/63 [==============================] - 2s 38ms/step - loss: 0.3650 - accuracy: 0.8930
Test Accuracy: 0.89300000667572